In [ ]:
%matplotlib inline


# Confusion matrix


Example of confusion matrix usage to evaluate the quality
of the output of a classifier on the iris data set. The
diagonal elements represent the number of points for which
the predicted label is equal to the true label, while
off-diagonal elements are those that are mislabeled by the
classifier. The higher the diagonal values of the confusion
matrix the better, indicating many correct predictions.

The figures show the confusion matrix with and without
normalization by class support size (number of elements
in each class). This kind of normalization can be
interesting in case of class imbalance to have a more
visual interpretation of which class is being misclassified.

Here the results are not as good as they could be as our
choice for the regularization parameter C was not the best.
In real life applications this parameter is usually chosen
using `grid_search`.




In [1]:
import os
os.environ['KERAS_BACKEND']='theano'
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import model_from_json
from keras.optimizers import rmsprop
#load the model
from keras.engine.topology import Layer, InputSpec
from keras import backend as K
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from pandas_ml import ConfusionMatrix


import pickle

CONTEXT_DIM = 100
class AttLayer(Layer):
    def __init__(self, regularizer=None, **kwargs):
        self.regularizer = regularizer
        self.supports_masking = True
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(name='W', shape=(input_shape[-1], CONTEXT_DIM), initializer='normal', trainable=True,
                                 regularizer=self.regularizer)
        self.b = self.add_weight(name='b', shape=(CONTEXT_DIM,), initializer='normal', trainable=True,
                                 regularizer=self.regularizer)
        self.u = self.add_weight(name='u', shape=(CONTEXT_DIM,), initializer='normal', trainable=True,
                                 regularizer=self.regularizer)
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.dot(K.tanh(K.dot(x, self.W) + self.b), self.u)
        ai = K.exp(eij)
        alphas = ai / K.sum(ai, axis=1).dimshuffle(0, 'x')
        if mask is not None:
            # use only the inputs specified by the mask
            alphas *= mask
        weighted_input = x * alphas.dimshuffle(0, 1, 'x')
        return weighted_input.sum(axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        config = {}
        base_config = super(AttLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_mask(self, inputs, mask):
        return None

# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
    
#---get--test-dataset--------------
f1 = open('x_test_file','rb')
x_test = pickle.load(f1)

f2 = open('y_test_labels','rb')
y_test = pickle.load(f2)

#------loaded_the model-------------
json_file = open('HAN1_rmsprop_L20.002.json','r')

loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json,custom_objects={'AttLayer':AttLayer()})
#load_weight
loaded_model.load_weights("HAN1_rmsprop_L20.02.h5")

print("loaded model from disk")

optimizer1 = rmsprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
loaded_model.compile(loss = 'categorical_crossentropy',optimizer=optimizer1,metrics=['acc'])

# score, acc = loaded_model.evaluate(x_test,y_test,batch_size=125,verbose=1)

# print('Test score:', score)
# print('Test accuracy:', acc)

y_pred_1 = loaded_model.predict(x_test,batch_size=125,verbose=1)


print(y_test.shape)
print(y_pred_1.shape)




Y_ = np.argmax(y_pred, axis = 1)
test = np.argmax(y_test, axis = 1)





# confusion_matrix = ConfusionMatrix(y_test,y_pred)

# confusion_matrix.plot(normalized=True)
# plt.show()

# confusion_matrix.print_stats()



Using Theano backend.


loaded model from disk
25000/25000 [==============================] - 688s 28ms/step
Test score: 0.378045729995
Test accuracy: 0.883520000577
